# Experiment on New York Times Dataset

In [1]:
%load_ext cython

In [2]:
%%cython -a 

import cython
import numpy as np
cimport numpy as np
from libc.math cimport pow, exp
from scipy.special.cython_special cimport psi


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
def LDASVI_cython(double[:,:] doc_cnt, int K, double alpha0, double gamma0, 
                      int MB, double eps, double kappa, double tau0, int epoch, int seed, int printcycle):
    cdef int D, vocab_size, epoch_len
    vocab_size = doc_cnt.shape[0] # number of words in vocabulary
    D = doc_cnt.shape[1] # number of documents
    epoch_len = D//MB
    np.random.seed(seed)
    cdef double[:,:] gamma = np.random.rand(vocab_size,K) # initialization of topics
    
    cdef long[:] sample_id
    cdef double[:,:] alpha
    cdef double diff
  
    cdef double[:] word_phi = np.empty(K)
    cdef double word_phi_sum
    cdef double[:] tmp_alpha
    cdef double[:,:] tmp_gamma
    cdef int i,j,k
    
    cdef double[:] psi_sum_gam
    cdef double[:] psi_sum_alpha
    
    for ep in range(epoch):
        order = np.random.permutation(D)
        for t in range(epoch_len):
            itr = ep * epoch_len + t
            if printcycle!=0 and itr%printcycle==0:
                print('starting iteration %i'%itr)
            sample_id = order[t*MB:(t+1)*MB]
            alpha = np.ones((K,MB)) # initialization of topic proportions
            diff = eps + 1
            # compute psi(sum(gamma, axis=0))
            psi_sum_gam = np.zeros(K)
            for k in range(K):
                for i in range(vocab_size):
                    psi_sum_gam[k] += gamma[i,k]
                psi_sum_gam[k] = psi(psi_sum_gam[k])
            '''E-Step: update local variational parameters(phi,alpha) till convergent'''
            while(diff>eps):
                diff = 0
                tmp_gamma = np.zeros((vocab_size,K))

                # compute psi(sum(alpha, axis=0))
                psi_sum_alpha = np.zeros(MB)
                for i in range(MB):
                    for k in range(K):
                        psi_sum_alpha[i] += alpha[k,i]
                    psi_sum_alpha[i] = psi(psi_sum_alpha[i])

                for i in range(MB):
                    tmp_alpha = np.ones(K) * alpha0
                    for j in range(vocab_size):
                        if doc_cnt[j,sample_id[i]] != 0:
                            word_phi_sum = 0
                            for k in range(K):
                                word_phi[k] = exp(psi(gamma[j,k]) - psi_sum_gam[k] + psi(alpha[k,i]) - psi_sum_alpha[i])
                                word_phi_sum += word_phi[k]
                            # normalize phi, update tmp_alpha, tmp_gamma
                            for k in range(K):
                                word_phi[k] /= word_phi_sum
                                tmp_alpha[k] += word_phi[k] * doc_cnt[j,sample_id[i]]
                                tmp_gamma[j,k] += word_phi[k] * doc_cnt[j,sample_id[i]]
                    # accumulate diff to decide local convergence
                    for k in range(K):         
                        diff += abs(tmp_alpha[k] - alpha[k,i])
                        alpha[k,i] = tmp_alpha[k]
                diff = diff / K / MB

            '''M-Step: update global variational parameters(gamma)'''                
            rho_t = pow(tau0+itr, -kappa)
            for i in range(vocab_size):
                for j in range(K):
                    gamma[i,j] = (1-rho_t)*gamma[i,j] + rho_t*(gamma0+tmp_gamma[i,j]*D/MB) 
    return gamma # no need to return alpha, since alpha only includes topic proportion of a mini-batch of documents

In [8]:
from sta663_project_lda.preprocessing.gen_nytdata import nytdata_generator
nytdata_generator()

total number of documents: 8447
vocabulary size: 3012
The generated nyt data is sucessfully saved in ./data/


In [3]:
nytdata_mat = np.load('./data/nytdata_mat.npy')
nytdata_voc = np.load('./data/nytdata_voc.npy')

In [4]:
training_params = {
    'K':25, # topic_num
    'alpha0':0.01, # alpha_prior
    'gamma0':0.01, # gamma_prior
    'MB':256, # minibatch_size
    'kappa':0.5, # learning_rate=(tau0+itr)^(-kappa)
    'tau0':256,
    'eps':1e-3, # convergence criteria for local updates
}

In [6]:
gamma = LDASVI_cython(nytdata_mat, **training_params, epoch=10, seed=0, printcycle=10)
gamma = np.array(gamma)

starting iteration 0
starting iteration 10
starting iteration 20
starting iteration 30
starting iteration 40
starting iteration 50
starting iteration 60
starting iteration 70
starting iteration 80
starting iteration 90
starting iteration 100
starting iteration 110
starting iteration 120
starting iteration 130
starting iteration 140
starting iteration 150
starting iteration 160
starting iteration 170
starting iteration 180
starting iteration 190
starting iteration 200
starting iteration 210
starting iteration 220
starting iteration 230
starting iteration 240
starting iteration 250
starting iteration 260
starting iteration 270
starting iteration 280
starting iteration 290
starting iteration 300
starting iteration 310


In [7]:
from sta663_project_lda.visualization.demo_topics import topic_viz
topic_viz(gamma, nytdata_voc, topk=10)

topic 1:
top-10 key words:
 ['building' 'car' 'open' 'house' 'street' 'area' 'mile' 'city' 'move'
 'foot']
distribution of top-10 key words:
 [ 0.01092392  0.0101069   0.00898515  0.00898346  0.00859649  0.0085165
  0.00795211  0.00781908  0.00703364  0.00679702]
topic 2:
top-10 key words:
 ['problem' 'program' 'require' 'number' 'provide' 'system' 'test' 'expert'
 'research' 'study']
distribution of top-10 key words:
 [ 0.0098503   0.00915376  0.0080487   0.00801     0.00709948  0.00698882
  0.00675572  0.00673969  0.00656856  0.00622023]
topic 3:
top-10 key words:
 ['official' 'government' 'force' 'military' 'states' 'kill' 'american'
 'war' 'attack' 'report']
distribution of top-10 key words:
 [ 0.01604739  0.01277841  0.0105575   0.00968583  0.00950303  0.0086117
  0.00843836  0.00835712  0.00807433  0.00687968]
topic 4:
top-10 key words:
 ['child' 'mother' 'family' 'father' 'woman' 'daughter' 'son' 'school'
 'parent' 'mrs']
distribution of top-10 key words:
 [ 0.03342097  0.028544